In [1]:
from llama_index.indices.query.query_transform.base import (
    DecomposeQueryTransform,
)
from llama_index import LLMPredictor
from llama_index.query_engine.transform_query_engine import (
    TransformQueryEngine,
)

from agent_builder import *

In [2]:
storage_dir = "./model_storage/"
data_dir = "./data/"

car_models = ["model_3", "model_y", "model_s"]

In [3]:
# Load all model documents
model_docs = {}
for model in car_models:
    model_docs[model] = SimpleDirectoryReader(
        input_files=[data_dir + f"{model}.pdf"]
    ).load_data()

In [4]:
doc_agents, indices, llm = build_document_agents(storage_dir=storage_dir, docs=model_docs)

### build_document_agents functions is taken from : https://github.com/run-llama/create_llama_projects/blob/main/multi-document-agent/README.md

In [5]:
doc_agents

{'model_3': <llama_index.agent.openai_agent.OpenAIAgent at 0x18d38355210>,
 'model_y': <llama_index.agent.openai_agent.OpenAIAgent at 0x18d3e3a3a00>,
 'model_s': <llama_index.agent.openai_agent.OpenAIAgent at 0x18d39ad2e60>}

In [6]:
index_summaries = {}
for model in car_models:
    # set summary text for city
    index_summaries[model] = (
        f"This content is from technical documents of Tesla {model}. "
        f"Use this index if you need to lookup specific facts about {model}.\n"
        "Do not use this index if you want to analyze multiple models."
    )

In [7]:
graph = ComposableGraph.from_indices(
    SimpleKeywordTableIndex,
    [index for _, index in indices.items()],
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50,
)

root_index = graph.get_index()
# set id of root index
root_index.set_index_id("compare_contrast")
root_index.summary = (
    "This index contains technical documents of multiple Tesla models. "
    "Use this index if you want to compare Tesla models. "
)


decompose_transform = DecomposeQueryTransform(
    LLMPredictor(llm=llm), verbose=True
)

custom_query_engines = {}
for index in indices.values():
    query_engine = index.as_query_engine()
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_metadata={"index_summary": index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
    
custom_query_engines[graph.root_id] = root_index.as_query_engine(
    retriever_mode="simple",
    response_mode="tree_summarize"
)

# define query engine
graph_query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)


### Graph query engine and the design of unified query framework is built upon the example in: https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/q_and_a/unified_query.html

In [8]:
all_tools = []
for model in doc_agents.keys():
    model_summary = (
        f"This content contains technical documents of Tesla {model}. Use"
        f" this tool if you want to answer any questions about {model}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=doc_agents[model],
        metadata=ToolMetadata(
            name=f"tool_{model}",
            description=model_summary,
        ),
    )
    all_tools.append(doc_tool)
      
model_summary = (
    f"This content contains technical documents of all Tesla models. Use"
    f" this tool if you want to compare the spesifications of any Tesla models. \n"
)
doc_tool = QueryEngineTool(
    query_engine=graph_query_engine,
    metadata=ToolMetadata(
        name=f"tool_compare_contrast",
        description=model_summary,
    ),
)
all_tools.append(doc_tool)


tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
# if obj_index doesn't already exist
if not os.path.exists(f"./{storage_dir}/top"):
    storage_context = StorageContext.from_defaults()
    obj_index = ObjectIndex.from_objects(
        all_tools, tool_mapping, VectorStoreIndex, storage_context=storage_context
    )
    storage_context.persist(persist_dir=f"./{storage_dir}/top")
    # TODO: don't access private property

else:
    # initialize storage context from existing storage
    storage_context = StorageContext.from_defaults(
        persist_dir=f"./{storage_dir}/top"
    )
    index = load_index_from_storage(storage_context)
    obj_index = ObjectIndex(index, tool_mapping)

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=5),
    system_prompt=""" \
        You are an agent designed to answer queries about Tesla cars. \
        Please always use the tools provided to answer a question. \
        Do not rely on prior knowledge.\

""",
    verbose=True
)

### node mapping and top agent building part is taken from : https://github.com/run-llama/create_llama_projects/blob/main/multi-document-agent/README.md

In [9]:
model_query_response = top_agent.query("What are the technical specifications of Model 3's suspension system?")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_model_3 with args: {
"input": "suspension system"
}
STARTING TURN 1
---------------

=== Calling Function ===
Calling function: vector_tool with args: {
"input": "suspension system"
}
Got output: The suspension system of the vehicle is described as independent, with a double wishbone setup in the front and a multi-link setup in the rear. It also includes coil springs and telescopic dampers.

STARTING TURN 2
---------------

Got output: The suspension system of the Model 3 is designed to provide a smooth and comfortable ride. It features an independent suspension with a double wishbone setup in the front and a multi-link setup in the rear. This configuration helps to improve handling and stability while also providing a comfortable ride.

The suspension system includes coil springs and telescopic dampers. The coil springs help to absorb bumps and uneven road surfaces, while the telescopic dampers help to co

In [14]:
print(model_query_response)

The suspension system of the Tesla Model 3 consists of an independent suspension with a double wishbone setup in the front and a multi-link setup in the rear. It is designed to provide a smooth and comfortable ride while also improving handling and stability. The system includes coil springs and telescopic dampers to absorb bumps and control the movement of the suspension. Overall, the suspension system of the Model 3 aims to provide a balance between comfort and performance for an enjoyable driving experience.


In [10]:
summary_query_response = top_agent.query("Summarize the technical spesification of Tesla model S that can be important for customers.")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_model_s with args: {
  "input": "summary"
}
STARTING TURN 1
---------------

=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "model_s"
}
Got output: The Model S owner's manual is a comprehensive guide that provides detailed information on various aspects of the vehicle. It covers topics such as the vehicle's features, configuration, market region, and software version. The manual includes instructions on how to access and adjust the climate controls on the touchscreen. It explains that the default setting is Auto, which maintains optimum comfort in most weather conditions. The manual also mentions that the climate control system is powered by the high voltage Battery and prolonged use may decrease driving range. Additionally, it provides a warning for individuals with limited pain sensation to exercise caution when using the climate control system and seat heaters. For more 

In [11]:
print(summary_query_response)

The technical specifications of the Tesla Model S that can be important for customers include:

1. Range: The Model S offers different range options depending on the battery configuration. The Long Range version has an estimated range of up to 412 miles (EPA) on a full charge, while the Performance version has a slightly lower range of up to 387 miles (EPA).

2. Performance: The Model S Performance version offers impressive acceleration, with a 0-60 mph time as low as 1.99 seconds. It also has a top speed of up to 200 mph.

3. Battery: The Model S is equipped with a high-capacity battery pack that provides power to the electric motors. The battery pack is designed to be durable and long-lasting, with a warranty that covers 8 years or 150,000 miles (whichever comes first).

4. Charging: The Model S supports fast charging using Tesla's Supercharger network, allowing for quick charging on long-distance trips. It is also compatible with other charging networks using the CCS (Combined Charg

In [21]:
comparison_query_response = top_agent.query("Compare and contrast the active safety features of Tesla model 3 and model y and model s.")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_compare_contrast with args: {
"input": "active safety features of Tesla model 3, model y, and model s"
}
> Current query: active safety features of Tesla model 3, model y, and model s
> New query: What are the active safety features of the Tesla Model 3?
> Current query: active safety features of Tesla model 3, model y, and model s
> New query: What are the active safety features of the Tesla Model 3?
> Current query: active safety features of Tesla model 3, model y, and model s
> New query: What are the active safety features of Tesla Model Y according to the technical documents?
> Current query: active safety features of Tesla model 3, model y, and model s
> New query: What are the active safety features of Tesla Model Y according to the technical documents?
> Current query: active safety features of Tesla model 3, model y, and model s
> New query: What are the active safety features of the Tesla Model S

In [22]:
print(comparison_query_response)

Here is a comparison of the active safety features of the Tesla Model 3, Model Y, and Model S:

Tesla Model 3:
- Traffic-Aware Cruise Control
- Autosteer
- Auto Lane Change
- Navigate on Autopilot
- Autopark
- Summon
- Smart Summon
- Traffic Light & Stop Sign Control
- Autosteer on City Streets (Full Self-Driving Beta)

Tesla Model Y:
- Electronic Stability Control
- Traction Control
- Trailer Mode (if equipped)
- Vehicle Hold
- Autopilot
- Traffic-Aware Cruise Control
- Autosteer
- Auto Lane Change
- Navigate on Autopilot
- Autopark
- Summon
- Smart Summon
- Traffic Light & Stop Sign Control
- Autosteer on City Streets (Full Self-Driving Beta)

Tesla Model S:
- Forward Collision Warning
- Automatic Emergency Braking
- Obstacle-Aware Acceleration
- Traffic-Aware Cruise Control
- Autosteer
- Auto Lane Change
- Navigate on Autopilot
- Autopark
- Summon
- Smart Summon
- Traffic Light & Stop Sign Control
- Autosteer on City Streets (Full Self-Driving Beta)

Please note that the availabilit